In [1]:
import math
import numpy as np

# Small studdy on approximation

In [2]:
size = 1000
cos_table = [int(math.cos(x / size) * size) for x in  range(0, math.floor(math.pi * 2 * size))]

print(f"{min(cos_table)=}, {max(cos_table)=}")
# cos_table

min(cos_table)=-999, max(cos_table)=1000


In [3]:

def sotf_cos(x):
    return cos_table[math.floor(x) % len(cos_table)]

def diff_soft_cos(x):
    value = sotf_cos(x * size) / size
    return value - round(math.cos(x), 5)

def print_diff(x):
    print(f"{float(x):<10.5}: {diff_soft_cos(x)}")


print_diff(0)
print_diff(math.pi / 2)
print_diff(math.pi)
print_diff(math.pi * 3)

"mean diff: ", np.mean([diff_soft_cos(x) for x in np.arange(0, math.pi * 2 * 4, 0.01)])

0.0       : 0.0
1.5708    : 0.0
3.1416    : 0.0010000000000000009
9.4248    : 0.0010000000000000009


('mean diff: ', -5.584725536992502e-06)

# Code generator

In [4]:
import math

size = 2000
cos_table = [int(math.cos(x / size) * size) for x in  range(0, math.floor(math.pi * 2 * size))]
sin_table = [int(math.sin(x / size) * size) for x in  range(0, math.floor(math.pi * 2 * size))]

In [6]:
from  pathlib import Path

h_src = Path("./src/includes/trigo.h")
c_src = Path("./src/trigo.c")

h_content = """
#ifndef _TRIGO_H_
#define _TRIGO_H_

#include <stdint.h>

typedef int16_t softfloat_t;

softfloat_t float_scale();
softfloat_t soft_cos(softfloat_t value);
softfloat_t soft_sin(softfloat_t value);

#endif // _TRIGO_H_
""".strip()

c_content = """
#include "trigo.h"
""".strip()

def append_table(name, values):
    global c_content
    c_content += f"""

static softfloat_t _{name}_table[] = {{
""".rstrip()

    for idx, value in enumerate(values):
        if idx > 0:
            c_content += ", "
        if idx % 15 == 0:
            c_content += "\n    "
        c_content += f"{value}"

    c_content += "};"

append_table("cos", cos_table)
append_table("sin", sin_table)

c_content += f"""

softfloat_t float_scale()
{{
    return {size};
}}
"""

c_content += f"""
softfloat_t soft_cos(softfloat_t value)
{{
    return _cos_table[value % {len(cos_table)}];
}}

softfloat_t soft_sin(softfloat_t value)
{{
    return _sin_table[value % {len(sin_table)}];
}}
"""


h_src.write_text(h_content)
c_src.write_text(c_content)

162784